In [1]:
# from alignn.data import get_train_val_loaders
import os
import sys
sys.path.append('..')
sys.path.append('/home/holywater2/2023/_Reproduce')
from repo_utils.data_utils import mp
import repo_utils.debbug_utils as deb

# crystals_dict_list = mp.load_json_sample(pdirname="dataset")
# print(crystals_dict_list[0].keys())

/compuworks/anaconda3/envs/nequip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
crystals_dict_list = mp.load(dirname="mp_megnet_sample01")
print(crystals_dict_list[0].keys())

[I] Loading the dataset...


100%|██████████| 6924/6924 [00:03<00:00, 1993.84it/s]

[I] Loading complete: the number of the loaded data is 6923
dict_keys(['id', 'desc', 'formula', 'e_hull', 'gap pbe', 'mu_b', 'elastic anisotropy', 'bulk modulus', 'shear modulus', 'atoms', 'e_form'])


In [3]:
from jarvis.core.atoms import Atoms
from tqdm import tqdm

In [4]:
aa = Atoms.ase_converter(Atoms.from_dict(crystals_dict_list[0]['atoms']))
dd = Atoms.from_dict(crystals_dict_list[0]['atoms'])

In [12]:
from ase import Atoms as AseAtoms

# AseAtoms(
#     symbols=self.elements,
#     positions=self.cart_coords,
#     pbc=pbc,
#     cell=self.lattice_mat,
# )

In [29]:
bb = AseAtoms(dd.elements,dd.cart_coords,pbc=True,cell=dd.lattice_mat,info={"target":10})

In [25]:
bb

Atoms(symbols='O64Si32', pbc=True, cell=[10.31189668, 12.72961385, 14.16308836], constraint=10)

In [33]:
crystals_dict_list[0]

{'id': 'mp-556088',
 'desc': [1772.21,
  1631.8521500000002,
  5.455,
  -7.332450000000001,
  0.0,
  11.157584999999997,
  2.5,
  308.230357,
  56.45007524454477,
  -866.9,
  3.2821000000000007,
  3.6944,
  35.6835,
  0.0,
  -71.01329,
  1036.47359668924,
  4.042999999999999,
  1646.31,
  0.0,
  30.3,
  87.5,
  34.65,
  0.0,
  -0.7,
  0.0,
  30.2,
  34.65,
  88.05,
  869.527,
  1.62515,
  -9.511865,
  60.30329,
  -12.04,
  10.632715000000001,
  14.884865000000001,
  0.5707025938965848,
  -2.316,
  9.0301,
  12.57075,
  12.3286,
  30.040691,
  23.41545,
  73.64029,
  0.03833376202516427,
  0.21000000000000002,
  3.0,
  28.00023325,
  177.83333333333331,
  3957.464105303181,
  31.745855421243746,
  -795.88671,
  3.0,
  13.86,
  137.4590356864804,
  870.9,
  1.10215,
  26.04245,
  26.098969696969696,
  0.23006060606060602,
  6.63294,
  10.884865000000001,
  1634.27,
  74.16329,
  6.8848650000000005,
  7.718865000000001,
  -2.4178499999999996,
  167.3593044,
  7249.076650000001,
  45279.21

In [63]:
ls = []
unq_symbol = set([])
for data in crystals_dict_list:
    atom_data = Atoms.from_dict(data['atoms'])
    unq_symbol.update(atom_data.elements)
    ase_atom = AseAtoms(symbols=atom_data.elements,
                        positions=atom_data.cart_coords,
                        pbc=True,
                        cell=atom_data.lattice_mat,
                        info={"energy":data['e_form']}
    )
    ls.append(ase_atom)

In [64]:
for sym in unq_symbol:
    print(" -",sym)

 - Bi
 - Zn
 - Sm
 - Ag
 - Li
 - Lu
 - Ca
 - Al
 - Tm
 - Hf
 - Ta
 - Cd
 - O
 - H
 - Pa
 - Dy
 - Yb
 - Os
 - Nd
 - Mg
 - Ce
 - Fe
 - Pt
 - Br
 - Cl
 - Pb
 - Au
 - Ho
 - C
 - Ga
 - Ac
 - Sc
 - S
 - W
 - Na
 - Rb
 - Ni
 - I
 - Sr
 - Ba
 - Pd
 - As
 - Cu
 - Eu
 - La
 - Tb
 - B
 - Er
 - In
 - P
 - Be
 - K
 - V
 - Ge
 - Hg
 - Cr
 - Xe
 - Pm
 - Gd
 - Ti
 - Tc
 - Sn
 - Se
 - Nb
 - Ru
 - Np
 - Si
 - Zr
 - Ir
 - Te
 - Pr
 - Re
 - Y
 - Tl
 - Rh
 - Sb
 - U
 - Co
 - Mn
 - Pu
 - He
 - Th
 - F
 - Cs
 - Mo
 - N


In [49]:
dd.update("A")

In [65]:
len(ls)

6923

In [36]:
from nequip.data.dataset import ASEDataset 

In [66]:
dd = ASEDataset.from_atoms_list(ls, extra_fixed_fields={"r_max": 4.5})

temp/atoms.xyz


In [9]:
dd[0]

AtomicData(atomic_numbers=[96, 1], cell=[3, 3], edge_cell_shift=[1752, 3], edge_index=[2, 1752], pbc=[3], pos=[96, 3], r_max=4.5)

In [10]:
""" Train a network."""
import logging
import argparse
import warnings

# This is a weird hack to avoid Intel MKL issues on the cluster when this is called as a subprocess of a process that has itself initialized PyTorch.
# Since numpy gets imported later anyway for dataset stuff, this shouldn't affect performance.
import numpy as np  # noqa: F401

from os.path import isdir
from pathlib import Path

import torch

from nequip.model import model_from_config
from nequip.utils import Config
from nequip.data import dataset_from_config
from nequip.utils import load_file
from nequip.utils.test import assert_AtomicData_equivariant
from nequip.utils.versions import check_code_version
from nequip.utils._global_options import _set_global_options
from nequip.scripts._logger import set_up_script_logger

In [11]:
dataset = dd

In [12]:
validation_dataset = None

In [13]:
from nequip.train.trainer import Trainer

trainer = Trainer(model=None, **dict(config))

NameError: name 'config' is not defined

In [ ]:
from nequip.utils.unittests.model_tests import BaseEnergyModelTests